In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.io import fits
import healpy as hp

In [3]:
kboltz=1.3806503e-23 #MKS
clight=299792458.0 #MKS
hplanck=6.626068e-34 #MKS
TCMB = 2.72548 #Kelvin
d2r = np.pi / 180.

In [4]:
lonc = 107.2
latc = 5.2

In [17]:
((90./60)**2 - (67./60)**2)

1.0030555555555556

In [5]:
wmapfiles = ['../externaldata/wmap_band_smth_imap_r9_9yr_K_v5.fits', \
             '../externaldata/wmap_band_smth_imap_r9_9yr_Ka_v5.fits',\
             '../externaldata/wmap_band_smth_imap_r9_9yr_Q_v5.fits', \
             '../externaldata/wmap_band_smth_imap_r9_9yr_V_v5.fits', \
             '../externaldata/wmap_band_smth_imap_r9_9yr_W_v5.fits']
wmapfreqs = np.array([22.71, 32.95, 40.65, 60.64, 93.44]) * 1e9

In [18]:
def wmap_calc(wmap, nu, inside=60., outside=90., radius=60.):
    wmapmap = hp.read_map(wmap, verbose=False) * 1.e-3
    x = np.copy(wmapmap)
    nside = hp.get_nside(x)
    vecc = hp.rotator.dir2vec(lonc, latc, lonlat=True)
    rmask = hp.query_disc(nside, vecc, (radius/60.)*d2r, inclusive=False)
    amaskout = set(hp.query_disc(nside, vecc, (outside/60.)*d2r, inclusive=False))
    amaskin = set(hp.query_disc(nside, vecc, (inside/60.)*d2r, inclusive=False))
    amask = np.array(list(amaskout.difference(amaskin)))
    X = hplanck * nu / (kboltz * TCMB) 
    kthermo_to_intensity = 2. * kboltz * (nu / clight)**2 * (X**2 * np.exp(X)) / (np.exp(X) - 1.)**2
    y = x * kthermo_to_intensity * hp.nside2pixarea(nside) * 1.e26 
    rdata = y[rmask]
    adata = y[amask]
    flux = np.sum(rdata - np.median(adata))
    rms = np.std(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
    print int(nu*1e-9), flux, np.sqrt(rms**2 + (0.03*flux)**2)

In [19]:
for k, wf in enumerate(wmapfiles):
    wmap_calc(wf, wmapfreqs[k], inside=67, outside=90, radius=60)

22 31.8368764744 1.30263687972
32 31.3203840584 1.28472920074
40 28.592130141 1.20302540388
60 25.8088666908 1.30079429895
93 34.4799875708 2.25968866322


In [21]:
planckfiles = ['../externaldata/LFI_SkyMap_030-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/LFI_SkyMap_044-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/LFI_SkyMap_070-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/HFI_SkyMap_143-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_217-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_353-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_545-field-Int_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_857-field-Int_2048_R2.02_full.fits']
planckfreqs = np.array([28.4, 44.1, 70.4, 143., 217., 353., 545., 857.]) * 1e9
planckbeams = np.array([32.3, 27.1, 13.3, 7.3, 5., 4.8, 4.7, 4.3])

In [26]:
def planck_calc(planck, nu, beam, inside=60., outside=90., radius=60.):
    planckmap = hp.read_map(planck, verbose=False)
    newbeam = np.sqrt(60.**2 - beam**2) / 60.
    planckmap = hp.ud_grade(hp.smoothing(planckmap, fwhm=1.*d2r, verbose=False), 512)
    #planckmap = hp.smoothing(hp.ud_grade(planckmap, 512), fwhm=newbeam*d2r, verbose=False)
    x = np.copy(planckmap)
    nside = hp.get_nside(x)
    vecc = hp.rotator.dir2vec(lonc, latc, lonlat=True)
    rmask = hp.query_disc(nside, vecc, (radius/60.)*d2r, inclusive=False)
    amaskout = set(hp.query_disc(nside, vecc, (outside/60.)*d2r, inclusive=False))
    amaskin = set(hp.query_disc(nside, vecc, (inside/60.)*d2r, inclusive=False))
    amask = np.array(list(amaskout.difference(amaskin)))
    X = hplanck * nu / (kboltz * TCMB) 
    kthermo_to_intensity = 2. * kboltz * (nu / clight)**2 * (X**2 * np.exp(X)) / (np.exp(X) - 1.)**2
    if nu < 400e9:
        y = x * kthermo_to_intensity * hp.nside2pixarea(nside) * 1.e26 
    else:
        y = x * 1e6 * hp.nside2pixarea(nside)
    rdata = y[rmask]
    adata = y[amask]
    flux = np.sum(rdata - np.median(adata))
    rms = np.std(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
    print int(nu*1e-9), flux, np.sqrt(rms**2 + (0.03*flux)**2)

In [27]:
for k, pf in enumerate(planckfiles):
    planck_calc(pf, planckfreqs[k], planckbeams[k], inside=67, outside=90, radius=60)

Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
28 27.8998438334 1.15152940024
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
44 25.4430834825 1.12141717128
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
70 24.9374150319 1.44216819046
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
143 90.3707794099 6.07673172698
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
217 430.854687606 24.4283889186
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
353 1804.99031718 98.7423046027
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
545 6021.91762403 308.172669826
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
857 19103.0625733 896.062273461
